In [18]:
import pandas as pd
import numpy as np

# Chunk Detection

In [52]:
from scapy.all import *
from math import *

GET_THRESH = 300 # bytes
DOWN_THRESH = 300  # bytes
VIDEO_CHUNK_GETSIZE = 700 # bytes
AUDIO_CHUNK_GETSIZE = 600 # bytes
NETINFO_NUM = 25

class Chunk():
    def __init__(self, start_time = 0, server_ip='',ttfb = 0, download_time = 0, slack_time = 0, get_size=0, chunk_size = 0, type="", protocol=""):
        self.start_time = start_time
        self.server_ip = server_ip
        self.ttfb = ttfb
        self.download_time = download_time
        self.slack_time = slack_time
        self.get_size = get_size
        self.chunk_size = chunk_size
        self.type = type
        self.protocol = protocol
    def __str__(self):
        return f"{self.start_time} {self.ttfb} {self.download_time} {self.slack_time} {self.get_size} {self.chunk_size} {self.type} {self.protocol}"
    def __repr__(self):
        return f"{self.start_time} {self.ttfb} {self.download_time} {self.slack_time} {self.get_size} {self.chunk_size} {self.type} {self.protocol}"


def isUplink(p):
    return p[IP].src.startswith('192.168.')

def detectAV(c):
    if abs(c.get_size-VIDEO_CHUNK_GETSIZE) > abs(c.get_size-AUDIO_CHUNK_GETSIZE):
        flag=0
    else:
        flag=1
    if c.chunk_size<=80*1024:
        flag=2
    return flag

def chunkDetection(filename):
    a = rdpcap(filename)
    meta_time = float(a[0].time)
    chunk = {}
    chunks = []
    downFlag = {}
    for p in a:
        if p.haslayer(IP):
            ipSrc=p[IP].src
            ipDst=p[IP].dst
            pLen=p[IP].len
            pHdr=p[IP].ihl*4
            ip_time=float(p.time)
            if isUplink(p) and pLen-pHdr > GET_THRESH:
                if ipDst in chunk:
                    chunk[ipDst].slack_time = ip_time - chunk[ipDst].download_time
                    avFlag=detectAV(chunk[ipDst])
                    if avFlag==0:
                        # chunk[ipDst].type='a'
                        chunks.append(chunk[ipDst])
                    elif avFlag==1:
                        # chunk[ipDst].type='v'
                        chunks.append(chunk[ipDst])
                    else:
                        chunk.pop(ipDst)
                        downFlag.pop(ipDst)
                chunk[ipDst] = Chunk(start_time=ip_time, get_size=pLen-pHdr, server_ip=ipDst)
                downFlag[ipDst] = False
            elif not isUplink(p) and pLen > DOWN_THRESH:
                if ipSrc in chunk:
                    if not downFlag[ipSrc]:
                        chunk[ipSrc].ttfb = ip_time
                        downFlag[ipSrc] = True
                    chunk[ipSrc].download_time = ip_time
                    chunk[ipSrc].chunk_size += pLen - pHdr
                    chunk[ipSrc].protocol = p.proto
    
    for c in chunk.values():
        avFlag=detectAV(c)
        if avFlag==0:
            # c.type='a'
            chunks.append(c)
        elif avFlag==1:
            # c.type='v'
            chunks.append(c)
    return meta_time, chunks

filename = 'RequetDataSet-master/A/PCAP_FILES/baseline_Jan17_exp_30.pcap'
meta_time, chunks=chunkDetection(filename)

In [15]:
print(chunk)

[1520313059.202668 1520313059.439442 1520313060.353278 0.00678706169128418 310 194840 v 17, 1520313060.360065 1520313060.385325 1520313061.821628 0.04106402397155762 683 194221 v 17, 1520313062.330472 1520313062.343013 1520313062.877641 0.0001430511474609375 685 215506 v 17, 1520313063.142399 1520313063.14423 1520313063.430298 0.012369871139526367 699 223216 v 17, 1520313063.442668 1520313063.466102 1520313063.663112 0.03597712516784668 621 82680 v 17, 1520313063.699089 1520313063.715391 1520313065.213543 0.009701013565063477 686 470989 v 17, 1520313065.223244 1520313065.261538 1520313065.995429 0.029619932174682617 638 210694 v 17, 1520313066.025049 1520313066.118452 1520313069.45236 0.002541065216064453 687 1560399 v 17, 1520313069.454901 1520313069.50511 1520313069.979387 1.1072678565979004 636 315414 v 17, 1520313071.086655 1520313071.115073 1520313074.160344 3.147207021713257 688 2007176 v 17, 1520313077.307551 1520313077.33479 1520313078.189515 4.121658802032471 623 457444 v 17, 

In [55]:
columns = ['start_time', 'server_ip', 'type', 'ttfb', 'download_time', 'get_size', 'chunk_size', 'rel_start_time', 'rel_end_time']
df_chunk=pd.DataFrame(columns=columns)


for c in chunks:
    start_time = c.start_time
    server_ip = c.server_ip
    type = c.type
    ttfb = c.ttfb - c.start_time
    download_time = c.download_time - c.start_time
    get_size = c.get_size
    chunk_size = c.chunk_size
    rel_start_time = start_time-meta_time
    rel_end_time = c.download_time - meta_time
    s = pd.Series([start_time, server_ip, type, ttfb, download_time, get_size, chunk_size, rel_start_time, rel_end_time], index=columns)
    df_chunk.loc[len(df_chunk)] = s

av_getsize_gap = df_chunk.get_size.mean()

for i in range(len(df_chunk)):
    df_chunk.loc[i, 'type'] = 'a' if df_chunk.loc[i, 'get_size'] < av_getsize_gap else 'v'

df_chunk.sort_values(by='start_time', inplace=True)
df_chunk.to_csv('results/chunk_detection/A/baseline_Jan17_exp_30.csv', index=False)

In [56]:
df_chunk

,start_time,server_ip,type,ttfb,download_time,get_size,chunk_size,rel_start_time,rel_end_time
6,1516208887.77837,172.217.10.97,a,0.102662,14.326276,453,143372,1.795876,16.122152
0,1516208887.883347,173.194.184.139,a,0.022888,3.10512,1247,564594,1.900853,5.005973
1,1516208891.158829,173.194.184.139,a,0.000315,0.312027,1255,284685,5.176335,5.488362
2,1516208891.471111,173.194.184.139,v,0.001053,0.226336,1363,309178,5.488617,5.714953
3,1516208891.697607,173.194.184.139,a,0.019482,1.072918,1256,1075220,5.715113,6.788031
...,...,...,...,...,...,...,...,...,...
70,1516209299.832551,173.194.184.139,v,0.032158,1.769261,1367,2071300,413.850057,415.619318
71,1516209311.078533,173.194.184.139,v,0.012034,1.299998,1367,2126409,425.096039,426.396037
72,1516209321.089191,173.194.184.139,v,0.021621,1.865273,1367,2081301,435.106697,436.97197
73,1516209327.833738,173.194.184.139,a,0.017624,0.082237,1260,115337,441.851244,441.933481


In [57]:
start_time = df_chunk['start_time'].min()
end_time = df_chunk['start_time'].max()

columns=['timestamp']+[str(i) for i in range(1,121)]

df_120features=pd.DataFrame(columns=columns)

for t_msec in range(int(start_time*1000), int(end_time*1000+100), 100):
    s = [t_msec]
    t = t_msec/1000
    for w in range(1, 21):
        period = w*10.0
        if t-period<start_time:
            total_number_of_chunks_v = -1
            avg_chunk_size_v = -1
            download_time_v = -1
            total_number_of_chunks_a = -1
            avg_chunk_size_a = -1
            download_time_a = -1
        else:
            total_number_of_chunks_v = df_chunk[(df_chunk['type']=='v') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)].shape[0]
            avg_chunk_size_v = df_chunk[(df_chunk['type']=='v') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)]['chunk_size'].mean()
            download_time_v = df_chunk[(df_chunk['type']=='v') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)]['download_time'].sum()
            total_number_of_chunks_a = df_chunk[(df_chunk['type']=='a') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)].shape[0]
            avg_chunk_size_a = df_chunk[(df_chunk['type']=='a') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)]['chunk_size'].mean()
            download_time_a = df_chunk[(df_chunk['type']=='a') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)]['download_time'].sum()
        s += [total_number_of_chunks_v, avg_chunk_size_v, download_time_v, total_number_of_chunks_a, avg_chunk_size_a, download_time_a]
    
    df_120features.loc[len(df_120features)] = s

df_120features.to_csv('results/120features/A/baseline_Jan17_exp_30.csv', index=False)

121


In [63]:
start_time = df_chunk['start_time'].min()
end_time = df_chunk['start_time'].max()

columns=['timestamp']+[str(i) for i in range(1,128)]

df_127features=pd.DataFrame(columns=columns)

for t_msec in range(int(start_time*1000), int(end_time*1000+100), 100):
    df_127features.loc[len(df_127features)] = [t_msec] + [0]*127

for w in range(1, 21):
    period = w * 10.0
    for i in range(len(df_127features)):
        t = df_120features.loc[i, 'timestamp']/1000
        h = 0
        if t-period < start_time:
            total_number_of_chunks_v = -1
            avg_chunk_size_v = -1
            download_time_v = -1
            total_number_of_chunks_a = -1
            avg_chunk_size_a = -1
            download_time_a = -1
        else:
            while t-period > df_chunk.loc[h, 'start_time']:
                h += 1
            total_number_of_chunks_v = df_chunk[h:i][(df_chunk['type']=='v')].shape[0]
            avg_chunk_size_v = df_chunk[h:i][(df_chunk['type']=='v')]['chunk_size'].mean()
            download_time_v = df_chunk[h:i][(df_chunk['type']=='v')]['download_time'].sum()
            total_number_of_chunks_a = df_chunk[h:i][(df_chunk['type']=='a')].shape[0]
            avg_chunk_size_a = df_chunk[h:i][(df_chunk['type']=='a')]['chunk_size'].mean()
            download_time_a = df_chunk[h:i][(df_chunk['type']=='a')]['download_time'].sum()
        df_127features.loc[i, str((w-1)*6+1)] = total_number_of_chunks_v
        df_127features.loc[i, str((w-1)*6+2)] = avg_chunk_size_v
        df_127features.loc[i, str((w-1)*6+3)] = download_time_v
        df_127features.loc[i, str((w-1)*6+4)] = total_number_of_chunks_a
        df_127features.loc[i, str((w-1)*6+5)] = avg_chunk_size_a
        df_127features.loc[i, str((w-1)*6+6)] = download_time_a
    
df_127features.to_csv('results/120features/A/test_baseline_Mar06_exp_120.csv', index=False)

/tmp/ipykernel_13340/2568071926.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  total_number_of_chunks_v = df_chunk[h:i][(df_chunk['type']=='v')].shape[0]
/tmp/ipykernel_13340/2568071926.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  avg_chunk_size_v = df_chunk[h:i][(df_chunk['type']=='v')]['chunk_size'].mean()
/tmp/ipykernel_13340/2568071926.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  download_time_v = df_chunk[h:i][(df_chunk['type']=='v')]['download_time'].sum()
/tmp/ipykernel_13340/2568071926.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  total_number_of_chunks_a = df_chunk[h:i][(df_chunk['type']=='a')].shape[0]
/tmp/ipykernel_13340/2568071926.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  avg_chunk_size_a = df_chunk[h:i][(df_chunk['type']=='a')]['chunk_size'].mean()
/tmp/ipykernel_13340/256807

KeyboardInterrupt: 